# Data cleaning
##  1. Concatenate Users
We have two important samples. One from January of 2021 and other for October of 2019. This script imports the rts made by each user in our sample, one by one, and later combine all the retweets in a pickle.

In [5]:
import os
import pandas as pd
import numpy as np
import glob
from tqdm import tqdm

In [8]:
path = r"../../../Data/"

In [9]:
os.listdir(path)

['Matrices',
 'nodes.csv',
 'Pickle',
 'Usuarios_V1',
 'Graphs',
 'Tweets_DataFrames']

### Import January 21' tweets
[comment]: <> (Duda #1. En el archivo original el path es "../../Data/Checks/28-06-23" pero ahí solo hay .pkl)

We identify two users with their file corrupted: Usuario_82383620 and Usuario_2526574133

In [ ]:
# We create an empty aux list that will store the tweets.
tweets_aux = []

In [ ]:
files_jan = glob.glob(os.path.join(path, "users_jan/*.csv"))

for file in tqdm(files_jan):
    tweets_aux.append(pd.read_csv(file))

In [ ]:
# Finally, the tweet dataframe is established and tweets_aux is deleted.  
tweets = pd.concat(tweets_aux)
del tweets_aux
tweets = tweets.sort_values('ID').reset_index(drop = True)

In [ ]:
# Store results
tweets.to_pickle(os.path.join(path, "users_jan/tweets_jan21.gzip"), compression = "gzip")

In [ ]:
del tweets

### Import October 19' tweets

We identify one user with their file corrupted: Usuario_1153335229597278208

In [ ]:
# We create an empty aux list that will store the tweets.
tweets_aux = []

In [ ]:
files_oct = glob.glob(os.path.join(path, 'users_oct_19/*.csv'))

for file in tqdm(files_oct):
    tweets_aux.append(pd.read_csv(file))

In [ ]:
# Finally, the tweet dataframe is established and tweets_aux is deleted.  
tweets = pd.concat(tweets_aux)
del tweets_aux
tweets = tweets.sort_values('ID').reset_index(drop = True)

In [ ]:
# Store results
tweets.to_pickle(os.path.join(path, "users_oct_19/tweets_oct19.gzip"), compression = "gzip")

In [ ]:
del tweets

### Import Usuarios_V1 tweets

In [10]:
# We create an empty aux list that will store the tweets.
tweets_aux = []

In [12]:
files_v1 = glob.glob(os.path.join(path, 'Usuarios_V1/*.csv'))

for file in tqdm(files_v1):
    tweets_aux.append(pd.read_csv(file))

  0%|          | 0/37324 [00:00<?, ?it/s]

 43%|████▎     | 15933/37324 [03:35<03:26, 103.51it/s]/tmp/ipykernel_53504/1028835383.py:4: DtypeWarning: Columns (6,8,10,17) have mixed types. Specify dtype option on import or set low_memory=False.
  tweets_aux.append(pd.read_csv(file))
 47%|████▋     | 17649/37324 [03:57<04:10, 78.55it/s] /tmp/ipykernel_53504/1028835383.py:4: DtypeWarning: Columns (18) have mixed types. Specify dtype option on import or set low_memory=False.
  tweets_aux.append(pd.read_csv(file))
 54%|█████▎    | 20026/37324 [04:30<03:46, 76.32it/s] /tmp/ipykernel_53504/1028835383.py:4: DtypeWarning: Columns (6,8,10,17) have mixed types. Specify dtype option on import or set low_memory=False.
  tweets_aux.append(pd.read_csv(file))
100%|██████████| 37324/37324 [08:23<00:00, 74.13it/s] 


In [13]:
# Finally, the tweet dataframe is established and tweets_aux is deleted.  
tweets = pd.concat(tweets_aux)
del tweets_aux
# tweets = tweets.sort_values('ID').reset_index(drop = True)

In [15]:
# Store results
# run sudo chmod 777 Data/Tweets_DataFrames in bash if it is needed
tweets.to_pickle(os.path.join(path, "Tweets_DataFrames/tweets_Usuarios_V1.gzip"), compression = "gzip")

In [16]:
del tweets

---
## 2. Assign political affiliation based on retweets
This section determines the political affiliation (left, center, right) of each user in our sample by analyzing the retweets they've made.

We use a list of political influencers that have been previously categorized as left, center, or right by La Silla Vacia, a Colombian news outlet. For each user, we tally the number of retweets (excluding those with comments) corresponding to each influencer. From this, we calculate the total number of tweets associated with each political category.

In [ ]:
# Let's load the id_to_party_map pickle file. This pickle contains some labels for political influencers in Colombia. 
# These were assigned using "La Silla Vacia" research.
mapa = pd.read_pickle("../Data/mapa.pkl")

In [ ]:
# Now we assign each RT a political label according to its influencer's label.
tweets["Party"] = np.nan
tweets.loc[tweets["Reference Type"] == "retweeted", "Party"] = tweets.loc[tweets["Reference Type"] == "retweeted",
                                                                         "Referenced Tweet Author ID"].map(mapa)

In [ ]:
# Store results
tweets.to_pickle("../Data/tweets_oct19_jan21.gzip", compression = "gzip")